<a href="https://colab.research.google.com/github/Sarang0218/Fourier_proof_of_concept/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pydub

In [9]:
!mkdir Cache

In [18]:
!ls

'1_NewJeans (뉴진스) '\''OMG'\'' Official MV (Performance ver.1)-sVTy_wmn5SU_(Instrumental).wav'
'Boogie Party.mp3'
 Cache
 drive
 log2FINISHIBLE.txt
 sample_data
 s.wav


In [17]:
# (1) 모듈 불러오기
from scipy.io import wavfile
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
from scipy.fftpack import fft, fftshift
import os
from pydub import AudioSegment
import math

# (2) wav 파일 나누기 (스택 오버플로우의 siddhantsomani 답변 참고)
class WavFileSplitter:
    def __init__(self, folder_path, file_name):
        self.folder_path = folder_path
        self.file_name = file_name
        self.file_path = os.path.join(folder_path, file_name)
        self.audio = AudioSegment.from_wav(self.file_path)

    def get_duration(self):
        return self.audio.duration_seconds * 1000

    def split_single(self, from_ms, to_ms, split_filename):
        start_time = from_ms
        end_time = to_ms
        split_audio = self.audio[start_time:end_time]
        split_audio.export(os.path.join(self.folder_path, 'Cache', split_filename), format="wav")

    def split_multiple(self, ms_per_split):
        total_ms = math.ceil(self.get_duration())
        split_number = 0
        for i in range(1, total_ms, ms_per_split):
            split_number += 1
            split_fn = f"{split_number}_{self.file_name}"
            self.split_single(i, i + ms_per_split, split_fn)

            if i == total_ms - ms_per_split:
                print('All split successfully')

# (3) 변수 설정
folder_path = ''
file_suffix = "s"
file_name = file_suffix+".wav"
beats_per_minute = 120
acceptor_threshold = 1  # 1 beat
splitter = WavFileSplitter(folder_path, file_name)
splitter.split_multiple(ms_per_split=int(60000 / beats_per_minute * acceptor_threshold))
amp_threshold = 360

# (4) 진동수에서 음이름으로 바꾸어주는 함수 구현
def hz_to_note(freq):
    notes = ['A', 'A#', 'B', 'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#']

    note_number = 12 * math.log2(freq / 440) + 49
    note_number = round(note_number)
    note = (note_number - 1 ) % len(notes)
    note = notes[note]

    octave = (note_number + 8 ) // len(notes)

    return note, octave

# (5) 파일을 푸리에 변환을 통해 분석하고 시각화
plt.rcParams['figure.dpi'] = 100
plt.rcParams['figure.figsize'] = (9, 7)

sound_dict = {}
length_in_seconds = 0
for file_name in os.listdir("Cache"):
    if ".wav" in file_name:
        sample_freq, sound = wavfile.read(os.path.join("Cache", file_name))
        sound = sound / 2.0**15
        length_in_seconds = sound.shape[0] / sample_freq
        time = np.arange(sound.shape[0]) / sound.shape[0] * length_in_seconds
        signal = sound[:,0]
        fft_spectrum = np.fft.rfft(signal)
        freq = np.fft.rfftfreq(signal.size, d=1. / sample_freq)

        fft_spectrum_abs = np.abs(fft_spectrum)
        #plt.plot(freq, fft_spectrum_abs)
        #plt.xlabel("Frequency, Hz")
        #plt.ylabel("Amplitude, units")
        #plt.show()

        note_list = []
        for i, f in enumerate(fft_spectrum_abs):
            if f > amp_threshold:  # Selecting representative notes

                knote = hz_to_note(np.round(freq[i],1))
                if knote not in note_list:
                    note_list.append(knote)

        sound_dict[file_name] = note_list


finishable_song = []
for k in range(len(sound_dict)):
    finishable_song.append([])
for i in range(1, len(sound_dict) + 1):
    for note in sound_dict[f"{i}_{file_suffix}.wav"]:
        finishable_song[i-1].append(note)


with open("log2FINISHIBLE.txt", "w") as log_file:
    log_file.write(str(sound_dict))


In [19]:
print(str(sound_dict))

{'180_s.wav': [('G#', 1), ('A', 1), ('A#', 1), ('B', 1), ('C', 2), ('C#', 2), ('D', 2), ('D#', 2), ('E', 2), ('F', 2), ('F#', 2), ('C', 3), ('C#', 3), ('D', 3), ('D#', 3), ('F', 3), ('A', 3), ('C', 4), ('D', 4), ('E', 4), ('F', 4), ('A', 4)], '68_s.wav': [('E', 1), ('G', 1), ('A', 1), ('A#', 1), ('B', 1), ('C', 2), ('C#', 2), ('D', 2), ('D#', 2), ('E', 2), ('F', 2), ('F#', 2), ('G', 2), ('G#', 2), ('A', 2), ('A#', 2), ('B', 2), ('C', 3), ('C#', 3), ('D', 3), ('D#', 3), ('F', 3), ('F#', 3), ('G', 3), ('G#', 3), ('A#', 3), ('C', 4), ('C#', 4), ('F', 4), ('A', 4)], '80_s.wav': [('F', 1), ('F#', 1), ('G', 1), ('G#', 1), ('A', 1), ('F#', 2), ('G', 2), ('D', 3), ('G', 3), ('A#', 3), ('D', 4), ('F', 4), ('A', 4)], '108_s.wav': [('F', 1), ('F#', 1), ('A', 1), ('B', 1), ('C', 2), ('C#', 2), ('D', 2), ('D#', 2), ('E', 2), ('F', 2), ('F#', 2), ('G', 2), ('G#', 2), ('A', 2), ('B', 2), ('E', 3), ('F', 3), ('A', 3), ('B', 3), ('C', 4), ('G', 4)], '184_s.wav': [('G', 1), ('G#', 1), ('A', 1), ('A#', 1